In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# reading csv file data
movie_data_orig = pd.read_csv("movies_metadata.csv")

# all column names
print(movie_data_orig.columns)

# removing irrelevant columns
df_cleaned = movie_data_orig.drop(["adult", "belongs_to_collection", "homepage", "id", "imdb_id", "original_language",
                                   "original_title", "overview", "poster_path", "production_countries", "release_date",
                                   "spoken_languages", "status", "tagline", "video" ], axis=1)
len(df_cleaned)       # 45466
# print(df_cleaned.head(5))

# taking into account only those movies where budget and/or revenue is greater than $100,000 (some of values in budget & revenue
# columns are 0, 1, 2, 3 etc which do not make any sense

df_cleaned.dtypes             # datatype of all columns in the dataframe; (budget is of "object" datatype instead of "int")
df_cleaned.describe()           # giving summary statistics of columns with dtype = float64
# budget column contains alpha-numeric characters, so need to fix it
df_cleaned['budget'] = df_cleaned['budget'].str.extract('(\d+)', expand=False)   # removing all non-numeric values from budget column

df_cleaned["budget"] = df_cleaned["budget"].astype(float).fillna(0.0)    # changing budget column from object to float
# df_cleaned.dtypes       # confirming that budget's dtype has changed to float64
# df_cleaned.describe()
df_cleaned = df_cleaned.loc[df_cleaned['budget'] > 100000]   # subsetting df to only movies with budget greater than $100,000

df_cleaned = df_cleaned.loc[df_cleaned['revenue'] > 1000]      # subsetting df to only movies with revenue greater than $1000


# creating our target/label column showing status i.e success/flop movie.
df_cleaned["status"] = df_cleaned["revenue"]/df_cleaned["budget"]
# df_cleaned.describe()
# df_cleaned.dtypes


# Our criteria for success is any value greater than 1 else flop
df_cleaned["New_status"] = np.nan      # creating a new empty column
df_cleaned["New_status"] = df_cleaned["New_status"].mask( df_cleaned["status"] > 1, 1)
df_cleaned["New_status"] = df_cleaned["New_status"].mask( df_cleaned["status"] <= 1, 0)


# # 1. converting (genre) json column to normal string column

df_cleaned['genres'] = df_cleaned['genres'].replace(np.nan,'{}',regex = True)
df_cleaned['genres'] = pd.DataFrame(df_cleaned['genres'].apply(eval))
# dividing all genres in a cell into separate cols/series, concating it to main df & then dropping the original "genres" column from df
df_cleaned = pd.concat([df_cleaned.drop(['genres'], axis=1), df_cleaned['genres'].apply(pd.Series)], axis=1)
# Removing all columns except the major genre type for each movie
df_cleaned.drop(df_cleaned.iloc[:, 11:18], inplace = True, axis = 1)
# creating separate series for "id" & "name" and concating it to main df
df_cleaned = pd.concat([df_cleaned.drop([0], axis=1), df_cleaned[0].apply(pd.Series)], axis=1)
df_cleaned.rename(columns = {'name' : 'Genre'}, inplace = True)   # renaming col
df_cleaned.drop(df_cleaned.iloc[:, 10:12], inplace = True, axis = 1)     # dropping extraneous cols
print(df_cleaned.columns)


# 2. converting (production_companies) json column to normal string column

df_cleaned['production_companies'] = df_cleaned['production_companies'].replace(np.nan,'{}',regex = True)
df_cleaned['production_companies'] = pd.DataFrame(df_cleaned['production_companies'].apply(eval))
df_cleaned = pd.concat([df_cleaned.drop(['production_companies'], axis=1), df_cleaned['production_companies'].apply(pd.Series)], axis=1)
df_cleaned.drop(df_cleaned.iloc[:, 11:36], inplace = True, axis = 1)
df_cleaned = pd.concat([df_cleaned.drop([0], axis=1), df_cleaned[0].apply(pd.Series)], axis=1)
df_cleaned.rename(columns = {'name' : 'Production Company'}, inplace = True)
df_cleaned.drop(df_cleaned.iloc[:, 10:12], inplace = True, axis = 1)
print(df_cleaned.columns)


# there are many entries where the number of people who voted for a movie are 1, 2 , 3 etc
df_cleaned = df_cleaned.loc[df_cleaned['vote_count'] > 100]      # subsetting df to only movies where atleast 100 people voted for a movie
len(df_cleaned)      # 3763

# Removing Duplicates
df_cleaned.drop_duplicates(inplace = True)
#print(len(df_cleaned)) # 3763
#print(df_cleaned)
df_cleaned.describe()
print(df_cleaned.dtypes)

print(df_cleaned.head())


C:\Users\Madhuri Yadav\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
Index(['budget', 'popularity', 'production_companies', 'revenue', 'runtime',
       'title', 'vote_average', 'vote_count', 'status', 'New_status', 'Genre'],
      dtype='object')
Index(['budget', 'popularity', 'revenue', 'runtime', 'title', 'vote_average',
       'vote_count', 'status', 'New_status', 'Genre', 'Production Company'],
      dtype='object')
budget                float64
popularity             object
revenue               float64
runtime               float64
title                  object
vote_average          float64
vote_count            float64
status                float64
N

C:\Users\Madhuri Yadav\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


NameError: name 'X' is not defined